# Texas Tow Trucks (`.apply` and `requests`)

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
import pandas as pd
import re
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

## Search for the TLDR Number `006565540C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [154]:
#url = "https://www.tdlr.texas.gov/tools_search/"
driver = webdriver.Chrome()
driver.get(url)

In [112]:
tldr_button = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/form/table/tbody/tr[5]/td[1]/input[1]").click()

In [113]:
tldr_entry = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/form/table/tbody/tr[5]/td[1]/input[2]")
tldr_entry.send_keys('006564940C')

In [114]:
submit = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/form/table/tbody/tr[10]/td/center/button").click()

In [115]:
details = driver.find_elements_by_tag_name('tbody')[4]
bus_name = details.find_elements_by_tag_name('tr')[1].text
bus_name = bus_name.replace("Name:", "").split("DBA")[0].strip()
print(bus_name)

A & N TOWING LLC


In [116]:
#Owner/operator
# Phone number
# License status (Active, Expired, Etc)
# Physical address

In [117]:
owner_op = details.find_elements_by_tag_name('tr')[2].text.replace("Owner/Officer:", "").strip()
ph_no = details.find_elements_by_tag_name('tr')[3].text.replace("Phone:", "").strip()

In [118]:
details2 = driver.find_elements_by_tag_name('tbody')[5]
details2_td = details2.find_elements_by_tag_name('td')
print(details2.text)

Certificate Information: Status:  Active
Number:  006564940C (Active) Carrier Type:  Tow Truck Company
Number of Active Tow Trucks:   1

Address Information
Mailing:
10838 DEEPWATER BAY
SAN ANTONIO, TX. 78251

Physical:
10838 DEEPWATER BAY
SAN ANTONIO, TX. 78251


In [119]:
print(details2_td[3].text)
lic_status = details2_td[1].text.replace("Status:", "").strip()
phys_add = re.findall(r'Physical:\s*(.*\n.*)', details2_td[3].text)[0]

Carrier Type:  Tow Truck Company
Number of Active Tow Trucks:   1

Address Information
Mailing:
10838 DEEPWATER BAY
SAN ANTONIO, TX. 78251

Physical:
10838 DEEPWATER BAY
SAN ANTONIO, TX. 78251


In [120]:
print(phys_add)

10838 DEEPWATER BAY
SAN ANTONIO, TX. 78251


In [100]:
tow_truck_df = pd.read_csv('tow-trucks.csv')
tow_truck_df.head()

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C
3,006542079C
4,0655733VSF


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [177]:
def tow_truck_machine(tdlr_no):
    url = "https://www.tdlr.texas.gov/tools_search/"
    driver = webdriver.Chrome()
    driver.get(url)
    tldr_button = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/form/table/tbody/tr[5]/td[1]/input[1]").click()
    tldr_entry = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/form/table/tbody/tr[5]/td[1]/input[2]")
    
#     for tldr_no in series:
    tldr_entry.send_keys(tdlr_no)
    submit = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/form/table/tbody/tr[10]/td/center/button").click()
    

    
    details = driver.find_elements_by_tag_name('tbody')[4]
    bus_name = details.find_elements_by_tag_name('tr')[1].text.replace("Name:", "").split("DBA")[0].strip()
    owner_op = details.find_elements_by_tag_name('tr')[2].text.replace("Owner/Officer:", "").strip()
    ph_no = details.find_elements_by_tag_name('tr')[3].text.replace("Phone:", "").strip()
    details2 = driver.find_elements_by_tag_name('tbody')[5]
    details2_td = details2.find_elements_by_tag_name('td')
    lic_status = details2_td[1].text.replace("Status:", "").strip()
    phys_add = re.findall(r'Physical:\s*(.*\n.*)', details2_td[3].text)[0]
    
    result = pd.Series([bus_name, owner_op, ph_no, lic_status, phys_add], index =['Name', 'Owner', 'Phone', 'License_status', 'Address'])
    return (result)


In [145]:
tow_sample = tow_truck_df.head()
tow_sample['TDLR Number'][2]

'006564940C'

In [160]:
tow_sample['link'] = tow_sample['TDLR Number'].apply(lambda x: "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber="+x)
tow_sample['link']

<ipython-input-160-31f54d9338ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tow_sample['link'] = tow_sample['TDLR Number'].apply(lambda x: "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber="+x)


0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006564940C
3    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006542079C
4    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0655733VSF
Name: link, dtype: object

In [161]:
tow_sample

,TDLR Number,link
0,006565540C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C
1,0654479VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF
2,006564940C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006564940C
3,006542079C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006542079C
4,0655733VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0655733VSF


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [158]:
pd.set_option('display.max_colwidth', None)

In [179]:
df_sample = tow_sample['TDLR Number'].apply(tow_truck_machine)
df_sample

,Name,Owner,Phone,License_status,Address
0,H & A TOWING LLC,HANEEN ABBAS MOHAMMEDAWI / MANAGER,512-999-8883,Active,"11710 JOSEPH CLAYTON DR\nAUSTIN, TX. 78753"
1,24/7 TOWING AND RECOVERY LLC,SCOTT JENSON / VICE PRESIDENT,Owner/Officer: CHRIS BRENES / PRESIDENT,Active,"3601 N COUNTY ROAD 1148\nMIDLAND, TX. 79705"
2,A & N TOWING LLC,ABDULRAHMAN Z ALABDULLAH / CEO,2106678546,Active,"10838 DEEPWATER BAY\nSAN ANTONIO, TX. 78251"
3,AR TOWING LLC,TANYA M JOHNSON / PRESIDENT,Owner/Officer: ROBERT J PETELL / OFFICER,Active,"3014 AURORA MIST DR\nLITTLE ELM, TX. 75068"
4,"BEST IN THE WEST TOWING, LLC",DANIEL L NASTANSKY / CEO,Owner/Officer: LINDA BRANUM / CEO,Insurance and/or fees not applied,"2801 N FREMONT AVE\nODESSA, TX. 79764"


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C`.

- *TIP: Use .apply and a function*
- *TIP: You'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', None)` to display aaaalll of the text in a cell*

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

### Re-open your dataframe to confirm you didn't save any extra weird columns

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**